### British Airways Project

## Overview 
The goal of this project is to learn about British Airways customers reviews of the airline. 
I will scrap a review website to look at how their customers reviewd the airline and try and 
gain insights to understand the customers. 

In [1]:
# import packages 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize 
import seaborn as sns



In [ ]:
base_url = 'https://www.airlinequality.com/airline-reviews/british-airways'
pages = 20
page_size = 100

reviews = []

for i in range(1, pages + 1):
    print(f'scrapping page {i}')
    
    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"
    
    # Collect HTML data from this page
    response = requests.get(url)
    
    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
        
        
    print(f"   ---> {len(reviews)} total reviews")


scrapping page 1


In [ ]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

In [ ]:
df.describe

In [ ]:
df.to_csv("BA_reviews.csv")

In [ ]:
for column in df:
    df['reviews'] = df['reviews'].map(lambda x: x.replace('✅ Trip Verified |',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('Not Verified |',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('✅ Verified Review | ',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('.',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace(',',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('"',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('?',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('!',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('(',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace(')',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('n\'t',' not'))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('-',' not'))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('’',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('get',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('would',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('one',''))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('ba','british-airways'))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('notairways','not airways'))
    df['reviews'] = df['reviews'].map(lambda x: x.replace('British Airways','british-airways'))
    df['reviews'] = df['reviews'].map(lambda x: x.lower())
    df.to_csv('BA_reviews_cleaned.csv',index=False)
    

In [ ]:
df.head()

In [ ]:
df = pd.read_csv('BA_reviews_cleaned.csv')

In [ ]:
df.head()

## EDA

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Create a list of revies from the DataFrame 
reviews_list = list(df['reviews'])
reviews_list

In [ ]:
# list to store the scores in
scores_combined = []
# itterate through the reviews to score the sentimate of the review. Print the review and the scores for a quick check. 
for reviews in reviews_list:
    scores = analyzer.polarity_scores(reviews)
    print(reviews)
    print(scores)
    # add the scores to the list
    scores_combined.append(scores)

In [ ]:
print(scores_combined)

In [ ]:
# create a DataFrame of the scores
df_scores = pd.DataFrame(scores_combined)
df_scores.head()

In [ ]:
# join the dataframe scores to the original dataframe
df = df.join(df_scores)
df.head()

In [ ]:
df.describe()

In [ ]:
df.describe

In [ ]:
def tokenize(column):
    """
    Tokenizes a Pandas dataframe column and returns a list of tokens.

    Args:
        column: Pandas dataframe column (i.e. df['text']).

    Returns:
        tokens (list): Tokenized list, i.e. [word, word, word]
    """

    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]

In [ ]:
df['tokenized'] = df.apply(lambda x: tokenize(x['reviews']), axis=1)
df[['tokenized']].head()

In [ ]:
df.head()

In [ ]:
fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2)
ax1.hist(x= df['neg'], color = 'red',)
ax1.set_title('Polarity Scores Negative')
ax2.hist(x=df['neu'],color = 'orange')
ax2.set_title('Polarity Scores Neutral')
ax3.hist(x=df['pos'],color = 'green')
ax3.set_title('Polarity Scores Positive')
ax4.hist(x=df['compound'],color = 'blue')
ax4.set_title('Polarity Scores Compound')
fig.suptitle('Sentiment Analysis Polarity Scores')
ax1.set_ylim(ymax=800)
ax1.set_xlim(xmax=1.0)
ax2.set_ylim(ymax=800)
ax2.set_xlim(xmax=1.0)
ax3.set_ylim(ymax=800)
ax3.set_xlim(xmax=1.0)
ax4.set_ylim(ymax=800)
ax4.set_xlim(xmax=1.0)
fig.tight_layout();

In [ ]:
df['compound'].mean()

In [ ]:
## this is where I am currently working. trying to make a frequency distribution of words. 

In [ ]:
# Tokenize the text in the 'text_column'
tokenized_text = [word.lower() for text in df['reviews'] for word in word_tokenize(str(text))]
# remove stop words
import nltk
stopwords = nltk.corpus.stopwords.words('english')
tokenized_cleaned = [x for x in tokenized_text if x not in stopwords]
print(tokenized_cleaned)


In [ ]:
from nltk.probability import FreqDist
# Calculate the frequency distribution using FreqDist
freq_dist = FreqDist(tokenized_cleaned)

# Print the most common words and their frequencies
print(freq_dist.most_common(20))

In [ ]:
# finding the frequency distinct in the tokens
# Importing FreqDist library from nltk and passing token into FreqDist
from nltk.probability import FreqDist
fdist = (FreqDist(tokenized_cleaned))
fdist

In [ ]:
all_fdist = pd.Series(dict(fdist.most_common(20)))
fig, ax =plt.subplots(figsize=(10,10))
all_plot = sns.barplot(x=all_fdist.index, y=all_fdist.values, ax=ax)
plt.xticks(rotation=90)
plt.xlabel('Words')
plt.ylabel('Count')
plt.title('20 Most Common Words in Reviews');